<a href="https://colab.research.google.com/github/csrviswa/Sentiment_Analysis/blob/master/Neighborhoods_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pip    
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
table = soup.find('table', class_='wikitable')
csv_file = open('postal_codes_of_Canada.csv', 'w')
csv_writer = csv.writer(csv_file)
coloumn = []
row = [];
for th in table.find_all('th'):
    coloumn.append(th.text.replace('\n', ''))
csv_writer.writerow(coloumn)
for td in table.find_all('td'):
    if(td.text.find('\n') != -1) :
       row.append(td.text.replace('\n', '')) 
       csv_writer.writerow(row)
       row.clear()
    else :
       row.append(td.text)
      
csv_file.close()    
data = pd.read_csv("postal_codes_of_Canada.csv") 
data.drop(data[data.Borough == 'Not assigned'].index, inplace=True)
data.reset_index(drop=True, inplace=True)
data = data.groupby(['Postcode','Borough'], sort=False).Neighbourhood.apply(', '.join).reset_index(name='Neighbourhood')
data.loc[data[data.Neighbourhood == 'Not assigned'].index, 'Neighbourhood'] = data.loc[data[data.Neighbourhood == 'Not assigned'].index, 'Borough']
print(data.shape)
data.head()


(103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [5]:
pd.set_option("display.precision", 7)
geo_data = requests.get('https://cocl.us/Geospatial_data')
with open('geo_data.csv', 'wb') as f:
    f.write(geo_data.content)
geo_df = pd.read_csv("geo_data.csv") 
geo_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
geo_df.head()
neighborhoods = pd.merge(data, geo_df, on=['Postcode'], how='inner')
neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7532586,-79.3296565
1,M4A,North York,Victoria Village,43.7258823,-79.3155716
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6542599,-79.3606359
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185180,-79.4647633
4,M7A,Queen's Park,Queen's Park,43.6623015,-79.3894938
